In [27]:
# Import Splinter, BeautifulSoup and dependencies
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from splinter import Browser
import time
import pandas as pd
from pprint import pprint

In [28]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# assign the met database to a variable name
db = mongo['kayak']

# assign the collection to a variable
flight_search = db['flight_search']


In [29]:
airport_code_test = pd.read_csv('C:\\Users\\gfoley\\OneDrive - Epiq Inc\\Desktop\\airport codes.csv')
airport_codes_df = pd.DataFrame(airport_code_test)
airport_codes_df

,IATA
0,LAX
1,MSP
2,ATL
3,SDF
4,HSV


In [32]:
#flight_search.delete_many({}) #empty the collection

DeleteResult({'n': 80, 'ok': 1.0}, acknowledged=True)

In [ ]:
#url = https://www.kayak.com/flights/{departure}-{destination}/{from_date}/{to_date}

In [33]:
# Complete loop for destination airports and kayak flights
airport_codes = airport_codes_df['IATA'].tolist()
departure = 'MCI' 

from_date = input("Enter departure date (YYYY-MM-DD): ")
to_date = input("Enter return date (YYYY-MM-DD): ")

browser = Browser('chrome')
trip_data = []

# Loop through each airport code
for airport in airport_codes:
    destination = airport
    url = f"https://www.kayak.com/flights/{departure}-{destination}/{from_date}/{to_date}"
    browser.visit(url)
    time.sleep(5)
    print(url)
    
    # Create a Beautiful Soup object
    soup = bs(browser.html,'html.parser')

    # Loop through HTML code, extract key metadata
    for trip_id, i in enumerate(soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare")):
        trip = {
            "trip_id": trip_id,
            "outbound_date": from_date,
            "return_date": to_date,
            "outbound_departure_time": "",
            "outbound_arrival_time": "",
            "outbound_flight_duration": "",
            "outbound_route": "",
            "outbound_airline": "",
            "return_departure_time": "",
            "return_arrival_time": "",
            "return_flight_duration": "",
            "return_route": "",
            "return_airline": "",
            "price": "",
            "tier": ""
        }
        
        # Extract outbound flight data
        outbound_flight_info = i.find_all('div', class_='VY2U')[0]
        outbound_flight_times = outbound_flight_info.find_all('span')
        trip["outbound_departure_time"] = outbound_flight_times[0].text.strip()
        trip["outbound_arrival_time"] = outbound_flight_times[2].text.strip()
        outbound_airline_name = outbound_flight_info.find('div', class_='c_cgF c_cgF-mod-variant-default')
        trip["outbound_airline"] = outbound_airline_name.text.strip()
        trip["outbound_flight_duration"] = i.find('div', class_='xdW8').find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
        trip["outbound_route"] = i.find('div', class_='xdW8').find('div', class_='EFvI').text.strip()
        
        # Extract return flight data
        return_flight_info = i.find_all('div', class_='VY2U')[1]
        return_flight_times = return_flight_info.find_all('span')
        trip["return_departure_time"] = return_flight_times[0].text.strip()
        trip["return_arrival_time"] = return_flight_times[2].text.strip()
        return_airline_name = return_flight_info.find('div', class_='c_cgF c_cgF-mod-variant-default')
        trip["return_airline"] = return_airline_name.text.strip()
        trip["return_flight_duration"] = i.find_all('div', class_='xdW8')[1].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
        trip["return_route"] = i.find_all('div', class_='xdW8')[1].find('div', class_='EFvI').text.strip()
        
        # Extract price and tier
        trip["price"] = i.find('div', class_="f8F1-price-text").text.strip()
        trip["tier"] = i.find('div', class_="aC3z-name aC3z-mod-ellipsis").text.strip()
        
        trip_data.append(trip)
        time.sleep(2)
flight_search.insert_many(trip_data)
browser.quit()

Enter departure date (YYYY-MM-DD):  2024-06-07
Enter return date (YYYY-MM-DD):  2024-06-10


https://www.kayak.com/flights/MCI-LAX/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-MSP/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-ATL/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-SDF/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-HSV/2024-06-07/2024-06-10


# Kayak Web Scrapping Tests and HTML Investigation

In [41]:
# kayak URL testing
airport_codes = airport_codes_df['IATA'].tolist()
departure = 'MCI' 

from_date = input("Enter departure date (YYYY-MM-DD): ")
to_date = input("Enter return date (YYYY-MM-DD): ")

# Loop through each airport code
for airport in airport_codes:
    destination = airport
    url = f"https://www.kayak.com/flights/{departure}-{destination}/{from_date}/{to_date}"
    print(url)


Enter departure date (YYYY-MM-DD):  2024-06-07
Enter return date (YYYY-MM-DD):  2024-06-10


https://www.kayak.com/flights/MCI-LAX/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-MSP/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-ATL/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-SDF/2024-06-07/2024-06-10
https://www.kayak.com/flights/MCI-HSV/2024-06-07/2024-06-10


In [44]:
# single URL test for web scraping elements
browser = Browser('chrome')
browser.visit('https://www.kayak.com/flights/MCI-SAN/2024-07-04/2024-07-11')
time.sleep(5)
# Create a Beautiful Soup object
soup = bs(browser.html,'html.parser')

In [46]:
# single URL test
# Find all flight text elements in HTML
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_ = "nrc6 nrc6-mod-pres-multi-fare"):
    print(i.text)

Best6:59 am – 12:55 pmSpirit Airlines1 stopLAS7h 56mMCI-SAN10:20 pm – 6:12 am+1Spirit Airlines1 stopLAS5h 52mSAN-MCI$201EconomyPricelineView Deal
Cheapest7:44 pm – 7:07 am+1Frontier1 stopDEN13h 23mMCI-SAN10:20 pm – 6:12 am+1Spirit Airlines1 stopLAS5h 52mSAN-MCI$197EconomyGotogateView Deal
6:59 am – 12:55 pmSpirit Airlines1 stopLAS7h 56mMCI-SAN8:47 pm – 6:12 am+1Spirit Airlines1 stopLAS7h 25mSAN-MCI$201EconomyPricelineView Deal
6:59 am – 1:56 pmSpirit Airlines1 stopLAS8h 57mMCI-SAN10:20 pm – 6:12 am+1Spirit Airlines1 stopLAS5h 52mSAN-MCI$201EconomyPricelineView Deal
6:59 am – 1:56 pmSpirit Airlines1 stopLAS8h 57mMCI-SAN8:47 pm – 6:12 am+1Spirit Airlines1 stopLAS7h 25mSAN-MCI$201EconomyPricelineView Deal
6:59 am – 12:55 pmSpirit Airlines1 stopLAS7h 56mMCI-SAN7:32 pm – 6:12 am+1Spirit Airlines1 stopLAS8h 40mSAN-MCI$201EconomyPricelineView Deal
6:59 am – 1:56 pmSpirit Airlines1 stopLAS8h 57mMCI-SAN7:32 pm – 6:12 am+1Spirit Airlines1 stopLAS8h 40mSAN-MCI$201EconomyPricelineView Deal
7:44 pm

In [14]:
# single URL test
# Find flight times in HTML
flight_times_list = []

for flight in soup.find_all('div', class_='VY2U'):  
    flight_times = flight.find_all('span')  
    departure_time = flight_times[0].text.strip()  
    arrival_time = flight_times[2].text.strip() 
    airline_name = flight.find('div', class_='c_cgF c_cgF-mod-variant-default').text.strip()

    flight_times_list.append({
        "departure_time": departure_time,
        "arrival_time": arrival_time,
        "airline": airline_name
    })

print(flight_times_list)

[{'departure_time': '5:45 am', 'arrival_time': '9:01 am', 'airline': 'Delta'}, {'departure_time': '6:15 am', 'arrival_time': '1:53 pm', 'airline': 'Delta'}, {'departure_time': '5:13 pm', 'arrival_time': '8:40 pm', 'airline': 'United Airlines'}, {'departure_time': '4:10 pm', 'arrival_time': '12:29 am+1', 'airline': 'United Airlines'}, {'departure_time': '7:44 pm', 'arrival_time': '7:07 am+1', 'airline': 'Frontier'}, {'departure_time': '10:20 pm', 'arrival_time': '6:12 am+1', 'airline': 'Spirit Airlines'}, {'departure_time': '5:59 am', 'arrival_time': '8:52 am', 'airline': 'American Airlines'}, {'departure_time': '3:31 pm', 'arrival_time': '11:20 pm', 'airline': 'American Airlines'}, {'departure_time': '5:59 am', 'arrival_time': '8:52 am', 'airline': 'American Airlines'}, {'departure_time': '4:20 pm', 'arrival_time': '11:50 pm', 'airline': 'American Airlines'}, {'departure_time': '2:22 pm', 'arrival_time': '6:08 pm', 'airline': 'American Airlines'}, {'departure_time': '3:31 pm', 'arrival

In [15]:
# single URL test
# Find flight durations in HTML
flight_durations_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    duration_route = i.find_all('div', class_='xdW8')
    outbound_flight_duration = duration_route[0].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
    return_flight_duration = duration_route[1].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
    flight_durations_list.append({
        "outbound_flight_duration": outbound_flight_duration,
        "return_flight_duration": return_flight_duration
    })

print(flight_durations_list)

[{'outbound_flight_duration': '5h 16m', 'return_flight_duration': '5h 38m'}, {'outbound_flight_duration': '5h 27m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '13h 23m', 'return_flight_duration': '5h 52m'}, {'outbound_flight_duration': '4h 53m', 'return_flight_duration': '5h 49m'}, {'outbound_flight_duration': '4h 53m', 'return_flight_duration': '5h 30m'}, {'outbound_flight_duration': '5h 46m', 'return_flight_duration': '5h 49m'}, {'outbound_flight_duration': '5h 46m', 'return_flight_duration': '5h 30m'}, {'outbound_flight_duration': '5h 52m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '5h 17m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '5h 27m', 'return_flight_duration': '5h 40m'}, {'outbound_flight_duration': '4h 56m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '6h 08m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '4h 53m', 'return_flight_duration': '5h 59m'}, {'outbound

In [17]:
# single URL test
# Find flight routes in HTML
flight_routes_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    duration_route = i.find_all('div', class_='xdW8')
    outbound_route = duration_route[0].find('div', class_='EFvI').text.strip()
    return_route = duration_route[1].find('div', class_='EFvI').text.strip()
    flight_routes_list.append({
        "outbound_route": outbound_route,
        "return_route": return_route
    })
print(flight_routes_list)

[{'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}]


In [18]:
# single URL test
# Find price information in HTML
price_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    # Extract all price text elements
    price_texts = i.find_all('div', class_="f8F1-price-text")
    
    for price_text in price_texts:
        price = price_text.text.strip()
        price_list.append(price)

print(price_list)

['$305', '$385', '$218', '$298', '$197', '$218', '$298', '$244', '$324', '$244', '$324', '$270', '$350', '$218', '$298', '$258', '$338', '$322', '$402', '$334', '$414', '$258', '$338', '$305', '$385', '$305', '$385', '$322', '$402', '$244', '$324', '$361', '$441']


In [19]:
# single URL test
# Find tier information in HTML
tier_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    # Extract all price text elements
    tier_texts = i.find_all('div', class_="aC3z-name aC3z-mod-ellipsis")
    
    for tier_text in tier_texts:
        tier = tier_text.text.strip()
        tier_list.append(tier)

print(tier_list)

['Basic Economy', 'Main Cabin', 'Basic Economy', 'Economy', 'Economy', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Economy', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Economy']


In [21]:
# single URL test
# Combine all text elements into one for loop
flight_data = {
    "flight_times": [],
    "flight_durations": [],
    "flight_routes": [],
    "prices": [],
    "tiers": []
}

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    for flight in soup.find_all('div', class_='VY2U'):  
        flight_times = flight.find_all('span')  
        departure_time = flight_times[0].text.strip()  
        arrival_time = flight_times[2].text.strip() 
        airline_name = flight.find('div', class_='c_cgF c_cgF-mod-variant-default')
        airline = airline_name.text.strip()
        flight_data["flight_times"].append({
            "departure_time": departure_time,
            "arrival_time": arrival_time,
            "airline": airline
        })
        
    duration_route = i.find_all('div', class_='xdW8')
    for dr in duration_route:
        outbound_flight_duration = duration_route[0].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
        return_flight_duration = duration_route[1].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
        flight_data["flight_durations"].append({
            "outbound_flight_duration": outbound_flight_duration,
            "return_flight_duration": return_flight_duration
        })

        outbound_route = duration_route[0].find('div', class_='EFvI').text.strip()
        return_route = duration_route[1].find('div', class_='EFvI').text.strip()
        flight_data["flight_routes"].append({
            "outbound_route": outbound_route,
            "return_route": return_route
        })
    
    # Extract all price text elements
    price_texts = i.find_all('div', class_="f8F1-price-text")
    
    for price_text in price_texts:
        price = price_text.text.strip()
        flight_data["prices"].append(price)

    tier_texts = i.find_all('div', class_="aC3z-name aC3z-mod-ellipsis")
    
    for tier_text in tier_texts:
        tier = tier_text.text.strip()
        flight_data["tiers"].append(tier)

pprint(flight_data)


{'flight_durations': [{'outbound_flight_duration': '5h 16m',
                       'return_flight_duration': '5h 38m'},
                      {'outbound_flight_duration': '5h 16m',
                       'return_flight_duration': '5h 38m'},
                      {'outbound_flight_duration': '5h 27m',
                       'return_flight_duration': '6h 19m'},
                      {'outbound_flight_duration': '5h 27m',
                       'return_flight_duration': '6h 19m'},
                      {'outbound_flight_duration': '13h 23m',
                       'return_flight_duration': '5h 52m'},
                      {'outbound_flight_duration': '13h 23m',
                       'return_flight_duration': '5h 52m'},
                      {'outbound_flight_duration': '4h 53m',
                       'return_flight_duration': '5h 49m'},
                      {'outbound_flight_duration': '4h 53m',
                       'return_flight_duration': '5h 49m'},
                      {'outbou

In [42]:
# single URL test
# Combine all text elements using one for loop and placing records into a list
single_url_scrape = []

for trip_id, i in enumerate(soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare")):
    trip = {
        "trip_id": trip_id,
        "outbound_departure_time": "",
        "outbound_arrival_time": "",
        "outbound_flight_duration": "",
        "outbound_route": "",
        "outbound_airline": "",
        "return_departure_time": "",
        "return_arrival_time": "",
        "return_flight_duration": "",
        "return_route": "",
        "return_airline": "",
        "price": "",
        "tier": ""
    }
    
    # Extract outbound flight data
    outbound_flight_info = i.find_all('div', class_='VY2U')[0]
    outbound_flight_times = outbound_flight_info.find_all('span')
    trip["outbound_departure_time"] = outbound_flight_times[0].text.strip()
    trip["outbound_arrival_time"] = outbound_flight_times[2].text.strip()
    outbound_airline_name = outbound_flight_info.find('div', class_='c_cgF c_cgF-mod-variant-default')
    trip["outbound_airline"] = outbound_airline_name.text.strip()
    trip["outbound_flight_duration"] = i.find('div', class_='xdW8').find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
    trip["outbound_route"] = i.find('div', class_='xdW8').find('div', class_='EFvI').text.strip()
    
    # Extract return flight data
    return_flight_info = i.find_all('div', class_='VY2U')[1]
    return_flight_times = return_flight_info.find_all('span')
    trip["return_departure_time"] = return_flight_times[0].text.strip()
    trip["return_arrival_time"] = return_flight_times[2].text.strip()
    return_airline_name = return_flight_info.find('div', class_='c_cgF c_cgF-mod-variant-default')
    trip["return_airline"] = return_airline_name.text.strip()
    trip["return_flight_duration"] = i.find_all('div', class_='xdW8')[1].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
    trip["return_route"] = i.find_all('div', class_='xdW8')[1].find('div', class_='EFvI').text.strip()
    
    # Extract price and tier
    trip["price"] = i.find('div', class_="f8F1-price-text").text.strip()
    trip["tier"] = i.find('div', class_="aC3z-name aC3z-mod-ellipsis").text.strip()
    
    single_url_scrape.append(trip)

In [43]:
# single URL test
kayak_df = pd.DataFrame(single_url_scrape)
kayak_df

,trip_id,outbound_departure_time,outbound_arrival_time,outbound_flight_duration,outbound_route,outbound_airline,return_departure_time,return_arrival_time,return_flight_duration,return_route,return_airline,price,tier
0,0,5:57 pm,11:01 pm,5h 04m,MCI-HSV,American Airlines,5:48 pm,11:07 pm,5h 19m,HSV-MCI,American Airlines,$310,Basic Economy
1,1,5:57 pm,11:01 pm,5h 04m,MCI-HSV,American Airlines,4:43 pm,10:57 pm,6h 14m,HSV-MCI,American Airlines,$310,Basic Economy
2,2,6:28 pm,11:39 pm,5h 11m,MCI-HSV,American Airlines,5:48 pm,11:07 pm,5h 19m,HSV-MCI,American Airlines,$529,Main Cabin
3,3,5:57 pm,11:01 pm,5h 04m,MCI-HSV,American Airlines,3:23 pm,9:18 pm,5h 55m,HSV-MCI,American Airlines,$529,Main Cabin
4,4,4:23 pm,11:01 pm,6h 38m,MCI-HSV,American Airlines,5:48 pm,11:07 pm,5h 19m,HSV-MCI,American Airlines,$310,Basic Economy
5,5,4:20 pm,9:28 pm,5h 08m,MCI-HSV,American Airlines,5:48 pm,11:07 pm,5h 19m,HSV-MCI,American Airlines,$644,Main Cabin
6,6,4:23 pm,9:46 pm,5h 23m,MCI-HSV,American Airlines,5:48 pm,11:07 pm,5h 19m,HSV-MCI,American Airlines,$644,Main Cabin
7,7,6:28 pm,11:39 pm,5h 11m,MCI-HSV,American Airlines,4:43 pm,10:57 pm,6h 14m,HSV-MCI,American Airlines,$529,Main Cabin
8,8,4:02 pm,10:07 pm,6h 05m,MCI-HSV,American Airlines,5:48 pm,11:07 pm,5h 19m,HSV-MCI,American Airlines,$594,Main Cabin
9,9,6:28 pm,11:39 pm,5h 11m,MCI-HSV,American Airlines,3:23 pm,9:18 pm,5h 55m,HSV-MCI,American Airlines,$678,Main Cabin


In [25]:
# single URL test
# Extract all flight text elements
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_ = "M_JD-large-display"):
    print(i.text)

$305Basic EconomyDelta
$385Main CabinDelta
$218Basic EconomyUnited Airlines
$298EconomyUnited Airlines
$197EconomyBooking.com
$218Basic EconomyAmerican Airlines
$298Main CabinAmerican Airlines
$244Basic EconomyAmerican Airlines
$324Main CabinAmerican Airlines
$244Basic EconomyAmerican Airlines
$324Main CabinAmerican Airlines
$270Basic EconomyAmerican Airlines
$350Main CabinAmerican Airlines
$218Basic EconomyUnited Airlines
$298EconomyUnited Airlines
$258Basic EconomyUnited Airlines
$338EconomyUnited Airlines
$322Basic EconomyUnited Airlines
$402EconomyUnited Airlines
$334Basic EconomyUnited Airlines
$414EconomyUnited Airlines
$258Basic EconomyUnited Airlines
$338EconomyUnited Airlines
$305Basic EconomyAmerican Airlines
$385Main CabinAmerican Airlines
$305Basic EconomyDelta
$385Main CabinDelta
$322Basic EconomyUnited Airlines
$402EconomyUnited Airlines
$244Basic EconomyAmerican Airlines
$324Main CabinAmerican Airlines
$361Basic EconomyUnited Airlines
$441EconomyUnited Airlines


In [40]:
# single URL test
# Extract booking URL
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_ = "nrc6 nrc6-mod-pres-multi-fare"):
        booking_link_tag = i.find('a', role='link', href="")
        trip["booking_url"] = 'https://www.kayak.com' + booking_link_tag['href']
print(booking_link_tag)   

None


In [ ]:
browser.quit()